Testing data positive feedback dynamic (20%)

In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')[:100]
all_3374 = np.load('../Data/npy/all_3374D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')[:100]
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')[:100]

In [4]:
#The number of followers for each YouTuber
YouTuber_followers = np.sum(user_following, axis=0)
print(YouTuber_followers)
over5 = 0
for num in YouTuber_followers:
    if num >= 5:
        over5+=1
print('The num of followers over 5:',over5)

[ 8  0  2 24  1  0  0 11 47  7 25 32 10  1 16  1  6 24  3  2  1  6  4  0
 39 39 10 16 27 22  0  2 17 15  0  2  0  8  2  1 28 16  3 33  9 24  0  3
  0  9  0  0  0  2 28  2  2 18  0  3 23  0  2 39  0  1  1 65  0  2 18 21
  7  1 47 14 19  1  3 66  3  0 28  3 20  0  6  0]
The num of followers over 5: 42


In [5]:
print('user_following shape ',user_following.shape)
print('all_3374 shape ',all_3374.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (100, 88)
all_3374 shape  (88, 3374)
user_category shape  (100, 17)
YouTuber_category shape  (88, 17)


In [6]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (100, 17)


In [7]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [8]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  21


In [9]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
avg_over10 = 0
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
        avg_over10+=num
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)
print('avg 10:',avg_over10/len(user_idx_over10))

over 8: 68
over 10: 56
over 12: 28
avg 10: 12.517857142857142


In [10]:
len(user_idx_over10)

56

# Training data and Testing data

In [11]:
test_amount = 10
yt_test_amount = 20

In [12]:
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
test_idx = random.sample(user_idx,test_amount)

In [13]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        if len(temp_t)*0.2 < 1:
            t_for_test = random.sample(temp_t,1)
            f_for_test  = random.sample(temp_f,yt_test_amount-1)
        else:
            if len(temp_t)*0.2 > 5:
                t_num = int(round(float(len(temp_t)*0.2)))
            else:      
                t_num = 5
            #t_num = int(round(float(len(temp_t)*0.2)))
            t_for_test = random.sample(temp_t,t_num)
            f_for_test  = random.sample(temp_f,yt_test_amount-t_num)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [14]:
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

12 6
6
21 11
10
15 8
7
14 7
7
11 6
5
5 3
2
19 10
9
10 5
5
11 6
5
11 6
5


In [15]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
avg

6.8

In [16]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 100
The length of train_f: 100
The length of test_t: 10
The length of test_f: 10


# Recommendation  Module

In [24]:
n = len(user_following)
m = 88  
k = 128
l = 3374
embedding_dims = 150

In [38]:
def training(): 
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()

    #use_true=init_list_of_objects(136)
    #use_test=init_list_of_objects(136)

    #train_pair_t=[] #positive feedback
    #train_pair_f=[] #negative feedback
    train_yes_id=[] 
    for q in range(5):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]
        
        
            sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            train_yes_id.append(sample) #sample全部丟進去
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #change
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
                #print('YouTuber_category ', YouTuber_category[sample[k]])
                #print('User_category ',user_category_norm[z])
            #print(yesr)
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            yes=np.array(yes)
        
            #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                #print(ta,'--> positive feedback')
            
                pos = sample.index(ta)
                #new_sample = np.delete(sample,[pos])
                #new_yes = np.delete(yes,[pos],axis=0)
                #new_r_3 = np.delete(r_3,[pos])
                new_sample = sample
                new_yes = yes
                new_r_3 = r_3
                #print(len(yes),len(new_yes))
                #print(yes)
                #print(new_yes)
            
            
            
                #ta=random.choice(train_t[z]) #ta is true positve photo
                #train_pair_t.append(ta)
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                #print('Image_1 shape ',image_1.shape)
                #train_f_sample = random.sample(train_f[z],20)
                #print('True:',train_t_sample,'Now:',ta)
                #print('False:',train_f_sample)
                for b in train_f_sample:
                    #print('likes:',ta,';Not likes:',b)
                    #b=random.choice(train_f[z])  #b is no feedback photo
                    #train_pair_f.append(b)
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    #print('Image_2 shape',image_2.shape)
            
                    #use_test[z].append(b)
                    _w1,_wu,_wy,_wa,_wv,_beta,_norm_par,_a_list,r3,_auc, _loss,_=sess.run([w1,wu,wy,wa,wv,beta,norm_par,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: new_yes , l_id:new_sample, l_id_len:[len(new_sample)],r:new_r_3,
                                        image_i:image_1,image_j:image_2})
                
                    #print(XUIJ)
                    #print('loss=',_loss)
                    #print('auc=',_auc)
                    #print('user positive negative',z,ta,b)
                    #print('u,vi,vj',_norm_par[:3])
                    #print('w1,wu,wy,wa,wv',_norm_par[3:8])
                    #print('beta',_norm_par[8:9])
                    #print('Embedding',_norm_par)
                
                    #print('alpha softmax:',r3)
                    #print('alpha list before softmax:',_a_list)
                    #print('---------------------------------------------------')
        
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
                #now1+=1
        
            #np.savez('../Data/latent_factor/YRM_up10/'+str(q)+'_'+str(z)+'.npz', User=U_history, YouTuber=Y_history)
    
        #print('mine:',xuij_auc/136)    
        #print('a_list_soft:',r3)
        #print("total_loss:-----------------", total_loss/length)
        #print("train_auc:-------------------", train_auc/length)
        #print('---------------------------------------------------------------------------------------------------')
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        #print('time:',time.time()-t0,' sec')
    #print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
    return U, Y, A, E, A1, Au, Ay, Aa, Av,B

In [39]:
par_weights = [0.01,0.001,0.0001,0.00001,0.000001]
beta_weights = [1,0.1,0.01]
Embedding_weights = [1,0.1,0.01]
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                """
                n: the number of users
                m: the number of YouTubers
                k: latent dims
                l: feature dims
                """
                tf.reset_default_graph()

                user = tf.placeholder(tf.int32,shape=(1,))
                i = tf.placeholder(tf.int32, shape=(1,))
                j = tf.placeholder(tf.int32, shape=(1,))

                #多少個auxliary 
                xf = tf.placeholder(tf.float32, shape=(None,l))
                l_id = tf.placeholder(tf.int32, shape=(None,))
                l_id_len = tf.placeholder(tf.int32,shape=(1,))
                r = tf.placeholder(tf.float32,shape=(None,))


                image_i = tf.placeholder(tf.float32, shape=(1,l))
                image_j = tf.placeholder(tf.float32, shape=(1,l))

                with tf.variable_scope("item_level"):
                    user_latent = tf.get_variable("user_latent", [n, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    item_latent = tf.get_variable("item_latent", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                    aux_item = tf.get_variable("aux_item", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    W1 = tf.get_variable("W1", [n, k],
                                                      initializer=tf.contrib.layers.xavier_initializer())
                    Wu = tf.get_variable("Wu", [n,k,k],
                                                      initializer=tf.contrib.layers.xavier_initializer())
                    Wy = tf.get_variable("Wy", [m,k,k],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wa = tf.get_variable("Wa", [n,k,k],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wv = tf.get_variable("Wv", [n,k,l],
                                                    initializer=tf.contrib.layers.xavier_initializer())



                    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                    ########## Error part, how to get auxisize dynamically
                    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                with tf.variable_scope('feature_level'):
                    embedding = tf.get_variable("embedding", [embedding_dims,l],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Beta = tf.get_variable("beta", [n,embedding_dims],
                                             # initializer=tf.contrib.layers.xavier_initializer())
                                                     initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                #lookup the latent factors by user and id
                u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                vi = tf.nn.embedding_lookup(item_latent, i) 
                vj = tf.nn.embedding_lookup(item_latent, j)

                w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
                #wu = tf.squeeze(Wu)
                wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
                wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
                #wa = tf.squeeze(Wa)
                wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
                #wv = tf.squeeze(Wv)
                wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

                beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                a_list=tf.Variable([])
                q = tf.constant(0)
                def att_cond(q,a_list):
                    return tf.less(q,l_id_len[0])
                def att_body(q,a_list):
                    xfi = tf.expand_dims(xf[q],0) #(1,l)

                    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
                            tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                            tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                            tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
                    q += 1
                    return q,  a_list

                _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                a_list_soft=tf.nn.softmax(a_list)


                aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                q = tf.constant(0)
                def sum_att_cond(q,aux_np):
                    return tf.less(q,l_id_len[0])

                def sum_att_body(q,aux_np):
                    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                    q += 1
                    return q, aux_np

                _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                """
                for q in range(3): #取q個auxliary item
                    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                """

                aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                #tf.print('aux attention:',aux_np)
                aux_np+=u #user_latent factor + sum (alpha*auxilary)
                aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                #矩陣中對應函數各自相乘
                # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                xuij = tf.subtract(xui,xuj)

                norm_par = [tf.reduce_sum(tf.multiply(u, u)),tf.reduce_sum(tf.multiply(vi, vi)),tf.reduce_sum(tf.multiply(vj, vj)),
                           tf.reduce_sum(tf.multiply(w1, w1)),tf.reduce_sum(tf.multiply(wu, wu)),tf.reduce_sum(tf.multiply(wy, wy)),
                           tf.reduce_sum(tf.multiply(wa, wa)),tf.reduce_sum(tf.multiply(wv,wv)),tf.reduce_sum(tf.multiply(beta,beta)),tf.reduce_sum(tf.multiply(embedding,embedding))]
                l2_norm = tf.add_n([
                            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),

                            0.0001 * tf.reduce_sum(tf.multiply(w1, w1)),
                            paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                            pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                            paru_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                            paru_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                            beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                            Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                          ])

                loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                auc = tf.reduce_mean(tf.to_float(xuij > 0))
                
                Ur, Yr, Ar, Er, A1r, Aur, Ayr, Aar, Avr,Br = training()
                np.savez('../Data/grid_search_weight/'+str(pary_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight)+'.npz', 
                         U=Ur, Y=Yr, A=Ar, E=Er, W1=A1r, Wu=Aur, Wy=Ayr, Wa=Aar, Wv=Avr,B=Br)
                print('Finish,',str(paru_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight))
    


Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_1_1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_1_0.1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_1_0.01
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.1_1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.1_0.1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.1_0.01
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.01_1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.01_0.1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.01_0.01_0.01_0.01
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iteraction: 3
Iteraction: 4
Finish, 0.001_0.001_1_1
Iteraction: 0
Iteraction: 1
Iteraction: 2
Iterac

KeyboardInterrupt: 

In [82]:
def testing(U, Y, A, E, A1, Au, Ay, Aa, Av,B):
    #print(A1)
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    max1 = 0
    maxu = 0
    maxy = 0
    maxa = 0
    maxv = 0
    min1 = 100000000000000000
    minu = 100000000000000000
    miny = 100000000000000000
    mina = 100000000000000000
    minv = 100000000000000000
    test_yes_id=[]
    for s in range(test_amount):
        print(s,test_idx[s])

        yes=[]
        sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        #sample=result_yes_id[now]
        test_yes_id.append(sample)
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            #print(test_idx[s])
            #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))

            #Observe each part in attention , below par are all (128,1)
            testW1 = np.sum(A1[test_idx[s]])
            #print(A1[test_idx[s]])
            WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
            WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
            WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
            WvVy = np.sum(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))
            print('The sum of each par -->\nw1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
            if testW1 > max1:
                max1 = testW1
            if testW1 < min1:
                min1 = testW1
            if WuUu > maxu:
                maxu = WuUu
            if WuUu < minu:
                minu = WuUu
            if WyYy > maxy:
                maxy = WyYy
            if WyYy < miny:
                miny = WyYy
            if WaAa > maxa:
                maxa = WaAa
            if WaAa < mina:
                mina = WaAa
            if WvVy > maxv:
                mxv = WvVy
            if WvVy < minv:
                minv = WvVy
            alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                                np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
        mul=np.zeros((1,128))
        print('alpha------------',alpha)
        print('softmax alpha--------------',softmax(alpha))
        for i in range(len(sample)):
            mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
    print(max1,maxu,maxy,maxa,maxv)
    print(min1,minu,miny,mina,minv)
    return RS

In [83]:
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/'
files = os.listdir(path)
#U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
for file in files:
    par_data = np.load(path+file)
    U = par_data['U']
    Y = par_data['Y']
    A = par_data['A']
    E = par_data['E']
    W1 = par_data['W1']
    Wu = par_data['Wu']
    Wy = par_data['Wy']
    Wa = par_data['Wa']
    Wv = par_data['Wv']
    B = par_data['B']
    RS = testing(U, Y, A, E, W1, Wu, Wy, Wa, Wv,B)

0 67
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: 0.0069017364 
WaA: 8.3433895e-07 
WvV: 1.2763706787969119e-05
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: -0.0026362638 
WaA: -1.7194218e-06 
WvV: 8.066619305332057e-06
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: -0.0022502714 
WaA: -8.265143e-07 
WvV: 4.462120588188263e-05
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: 0.0020262771 
WaA: 5.7877264e-07 
WvV: 2.4405252545179113e-05
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: 0.006587241 
WaA: 4.9245267e-07 
WvV: 9.982960242636487e-06
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: -0.0017511599 
WaA: 9.647463e-07 
WvV: 3.4370452933431366e-05
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: 0.0025208867 
WaA: -1.0089749e-06 
WvV: 2.004008040684712e-05
The sum of each par -->
w1: -2.0262864e-11 
WuU: -3.2341426e-14 
wyY: -0

9 11
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: 0.00022794234 
WaA: 3.189983e-05 
WvV: -0.0007866912259144009
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: -0.0022502714 
WaA: 3.8947273e-06 
WvV: -0.0008614756861668125
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: 0.0007875436 
WaA: 7.1798645e-06 
WvV: -0.001077889109805357
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: 0.0031344697 
WaA: -2.4137944e-06 
WvV: -0.0008057326538910779
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: -0.004151799 
WaA: 1.6757152e-05 
WvV: -0.0009750162467399638
The sum of each par -->
w1: -6.717404e-08 
WuU: -8.751633e-08 
wyY: 0.006587241 
WaA: -3.2002026e-06 
WvV: -0.0009645609241451982
alpha------------ [5.35383502e-11 7.13631090e-13 6.58281185e-11 1.49819827e-11
 5.33274408e-12 1.99303131e-11]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
2.16861

9 11
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: 0.0058579613 
WaA: 6.7373905e-05 
WvV: -0.0009872185733805583
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: 0.0015239062 
WaA: 1.3140458e-05 
WvV: -0.0006392382149286053
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: -0.00039300555 
WaA: 2.1784537e-05 
WvV: -0.0010746493772788343
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: 0.0023714567 
WaA: -4.619711e-06 
WvV: -0.000287719672275999
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: -0.0007504354 
WaA: -1.4594124e-05 
WvV: -0.001091635788426466
The sum of each par -->
w1: -1.829926e-07 
WuU: -1.2173302e-08 
wyY: -0.00094337156 
WaA: 6.746137e-06 
WvV: -0.0007801453755415497
alpha------------ [-2.64055790e-10  5.08936490e-11  2.55536082e-11 -1.46644962e-11
 -2.73887031e-12 -6.04437877e-11]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.166

9 11
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: -0.003943553 
WaA: 1.7029417e-06 
WvV: -0.0004155076173557524
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: -0.002405857 
WaA: 2.1721526e-06 
WvV: -0.0007096674122439505
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: 0.00013812017 
WaA: 5.2659743e-05 
WvV: -0.00026170471961169784
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: 0.00075502857 
WaA: 5.897357e-05 
WvV: -0.0004025915918656091
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: 0.009698839 
WaA: 3.5536126e-05 
WvV: -0.0004621727036950476
The sum of each par -->
w1: -4.825899e-07 
WuU: 9.0545547e-07 
wyY: 0.0025076934 
WaA: -1.2265595e-05 
WvV: -0.00040592139949110676
alpha------------ [-4.01980508e-12 -9.82795484e-11 -2.45113734e-11  6.13170055e-13
  7.09884926e-11 -1.54505770e-10]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]

8 17
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: 0.008335121 
WaA: -2.2640183e-05 
WvV: 0.0003576879113661531
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -4.945e-10 
WaA: -3.876991e-06 
WvV: 0.0005206308357441703
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -0.0018047645 
WaA: -7.0869623e-07 
WvV: 0.0005846073869295279
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -0.0003814171 
WaA: 1.202844e-06 
WvV: 0.000393535733348454
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -0.005745962 
WaA: -1.090427e-05 
WvV: 0.00039652717846905603
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -0.00072620343 
WaA: 4.857662e-06 
WvV: 0.0005535173332873996
The sum of each par -->
w1: 2.6892457e-09 
WuU: -8.646135e-07 
wyY: -0.0019528766 
WaA: -5.8814844e-06 
WvV: 0.0004116129099831378
alpha------------ [ 0.00000000e+00 -1.27860276e-13 -2.40706957e-12 -5.36677561e-14
  1.15

...

6 63
The sum of each par -->
w1: 0.1174988 
WuU: -0.00045049674 
wyY: 5.7217953e-28 
WaA: 0.0006614018 
WvV: 0.06086926593525825
The sum of each par -->
w1: 0.1174988 
WuU: -0.00045049674 
wyY: -1.9608811e-07 
WaA: 0.0017362162 
WvV: 0.03568291872114823
alpha------------ [2.25961917e-05 8.08924976e-06]
softmax alpha-------------- [0.50000363 0.49999637]
7 77
The sum of each par -->
w1: 4.199691e-06 
WuU: -1.2072497e-06 
wyY: -4.531869e-27 
WaA: 6.967288e-05 
WvV: 0.0014927940304349577
The sum of each par -->
w1: 4.199691e-06 
WuU: -1.2072497e-06 
wyY: -2.6822908e-27 
WaA: 4.025236e-05 
WvV: 0.0020112773958772456
The sum of each par -->
w1: 4.199691e-06 
WuU: -1.2072497e-06 
wyY: 3.444131e-27 
WaA: 1.8469573e-05 
WvV: 0.0027296046591840026
The sum of each par -->
w1: 4.199691e-06 
WuU: -1.2072497e-06 
wyY: -2.2267065e-18 
WaA: 0.000118758486 
WvV: 0.001539682202241561
The sum of each par -->
w1: 4.199691e-06 
WuU: -1.2072497e-06 
wyY: -2.6939301e-28 
WaA: -5.2829437e-06 
WvV: 0.00067532

6 63
The sum of each par -->
w1: -0.00051474944 
WuU: 0.0005031773 
wyY: 1.9324843e-27 
WaA: 0.0005655126 
WvV: -0.018981138205502106
The sum of each par -->
w1: -0.00051474944 
WuU: 0.0005031773 
wyY: -2.5825665e-07 
WaA: 0.00021916034 
WvV: -0.032805286297661496
alpha------------ [1.46930118e-05 6.05682289e-07]
softmax alpha-------------- [0.50000352 0.49999648]
7 77
The sum of each par -->
w1: 5.411241e-06 
WuU: 2.0549996e-06 
wyY: 1.3105743e-27 
WaA: 8.377697e-05 
WvV: 0.0031457236538874594
The sum of each par -->
w1: 5.411241e-06 
WuU: 2.0549996e-06 
wyY: 1.3666955e-18 
WaA: -1.5064685e-05 
WvV: 0.002748916222270238
The sum of each par -->
w1: 5.411241e-06 
WuU: 2.0549996e-06 
wyY: 6.848966e-27 
WaA: 1.4425466e-05 
WvV: 0.002988594842263914
The sum of each par -->
w1: 5.411241e-06 
WuU: 2.0549996e-06 
wyY: 4.023517e-27 
WaA: 2.2259832e-05 
WvV: 0.0029808820212144497
The sum of each par -->
w1: 5.411241e-06 
WuU: 2.0549996e-06 
wyY: 5.5831792e-27 
WaA: -4.0329523e-05 
WvV: 0.002257

6 63
The sum of each par -->
w1: -0.0003989339 
WuU: 8.388615e-05 
wyY: -3.6133335e-27 
WaA: 8.585723e-05 
WvV: 0.09316354531880863
The sum of each par -->
w1: -0.0003989339 
WuU: 8.388615e-05 
wyY: -1.7922261e-07 
WaA: 0.0018858758 
WvV: 0.045942730438196105
alpha------------ [ 2.26857138e-05 -9.30896912e-07]
softmax alpha-------------- [0.5000059 0.4999941]
7 77
The sum of each par -->
w1: 9.611277e-06 
WuU: 7.7545025e-07 
wyY: 8.8184793e-29 
WaA: -1.0191568e-05 
WvV: 0.00033876646278225143
The sum of each par -->
w1: 9.611277e-06 
WuU: 7.7545025e-07 
wyY: -3.0221958e-18 
WaA: 5.1316656e-05 
WvV: 0.0001994211799547232
The sum of each par -->
w1: 9.611277e-06 
WuU: 7.7545025e-07 
wyY: -2.4320297e-28 
WaA: 0.000103068705 
WvV: 0.0001359695390679217
The sum of each par -->
w1: 9.611277e-06 
WuU: 7.7545025e-07 
wyY: -3.5539715e-28 
WaA: 1.3562305e-05 
WvV: -6.896750844188155e-05
The sum of each par -->
w1: 9.611277e-06 
WuU: 7.7545025e-07 
wyY: 5.2439174e-27 
WaA: 7.648555e-05 
WvV: 3.02

6 63
The sum of each par -->
w1: 0.10935195 
WuU: 0.0003416735 
wyY: -2.5083327e-27 
WaA: 7.6993485e-05 
WvV: -0.023924181064218418
The sum of each par -->
w1: 0.10935195 
WuU: 0.0003416735 
wyY: -7.745905e-07 
WaA: -0.0016915279 
WvV: -0.03904860073152445
alpha------------ [ 2.01523894e-05 -1.99531828e-06]
softmax alpha-------------- [0.50000554 0.49999446]
7 77
The sum of each par -->
w1: 1.0278245e-07 
WuU: -3.0323577e-06 
wyY: -3.2403498e-19 
WaA: -7.394275e-05 
WvV: -0.002004375636298913
The sum of each par -->
w1: 1.0278245e-07 
WuU: -3.0323577e-06 
wyY: 1.3505518e-27 
WaA: -4.07122e-05 
WvV: -0.0028388023521054868
The sum of each par -->
w1: 1.0278245e-07 
WuU: -3.0323577e-06 
wyY: -3.840761e-27 
WaA: -7.871679e-05 
WvV: -0.001833506541978567
The sum of each par -->
w1: 1.0278245e-07 
WuU: -3.0323577e-06 
wyY: 4.774894e-27 
WaA: 0.00015395152 
WvV: -0.002554651453958406
The sum of each par -->
w1: 1.0278245e-07 
WuU: -3.0323577e-06 
wyY: -1.973213e-27 
WaA: -7.0817056e-05 
WvV: 

5 24
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: 3.122703e-22 
WaA: 7.2442117e-06 
WvV: -0.00012069533590199815
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: -3.1448478e-15 
WaA: -6.985997e-07 
WvV: 1.968839823547216e-05
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: 6.4389227e-27 
WaA: -3.1398395e-07 
WvV: -0.00023637184856259492
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: 2.0245963e-18 
WaA: -1.8958249e-06 
WvV: -9.493689596032977e-05
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: 1.0453262e-09 
WaA: -7.77631e-06 
WvV: -6.875554772539885e-05
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: 1.0692856e-14 
WaA: 1.4289373e-06 
WvV: -5.7829117838223356e-05
The sum of each par -->
w1: -3.6084554e-08 
WuU: -2.5284265e-07 
wyY: -0.007331515 
WaA: 3.8805556e-06 
WvV: -0.00016436876855854154
alpha------------ [-6.95541273e-13 -1.11626431e-13 -5.8014432

# Get latent factor and Each weight

In [70]:
U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])

In [71]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)
print('Embedding shape:',E.shape)

User latent shape:  (100, 128)
photo latent shape:  (88, 128)
Auxilary latent shape:  (88, 128)
W1 weight shape:  (100, 128)
Wu weight shape: (100, 128, 128)
Wy weight shape: (88, 128, 128)
Wa weight shape: (100, 128, 128)
Wv weight shape: (100, 128, 3374)
Beta shape: (100, 150)
Embedding shape: (150, 3374)


array([[ 1.28755188e-13, -1.59495830e-08,  1.67970065e-08, ...,
        -6.79700233e-14, -1.15951595e-08,  2.96662903e-11],
       [ 7.16980017e-11, -1.62418325e-11, -1.24057414e-11, ...,
         8.91374561e-12, -2.79513977e-12, -1.84613626e-11],
       [-8.25461731e-18,  1.03062558e-11,  9.12969830e-18, ...,
         1.62189845e-11,  4.00709840e-14, -1.09470229e-20],
       ...,
       [-1.16417351e-24,  8.37993319e-26,  1.97979825e-19, ...,
         6.30885467e-31, -7.52699314e-31,  4.53611797e-30],
       [ 6.38316104e-22,  1.14164738e-17, -5.85400740e-23, ...,
        -2.08511815e-21, -1.43180177e-18,  3.81385353e-18],
       [ 2.83457591e-18, -5.11021429e-18,  1.61269826e-12, ...,
         1.17598116e-15,  1.54162794e-12,  2.98273906e-11]], dtype=float32)

# Testing Part

In [87]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        
        #Observe each part in attention , below par are all (128,1)
        testW1 = np.sum(np.multiply(A1[test_idx[s]],A1[test_idx[s]]))
        WuUu = np.sum(np.multiply(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T),np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)))
        WyYy = np.sum(np.multiply(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T),np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)))
        WaAa = np.sum(np.multiply(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T),np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T)))
        WvVy = np.sum(np.multiply(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T),np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T)))
        print('w1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
        
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
#print(RS[s])

0 50
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 1.5945054e-05 
WaA: 3.1537473e-21 
WvV: 2.2159012677821794e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 6.469689e-05 
WaA: 3.5369395e-21 
WvV: 1.8931885856131214e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 1.1103042e-05 
WaA: 3.3975095e-21 
WvV: 2.0774474670348896e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 2.2845748e-09 
WaA: 2.3888677e-21 
WvV: 2.0544184028555723e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 4.354328e-05 
WaA: 3.317824e-21 
WvV: 2.6350840712423046e-18
alpha------------ [ 4.12867649e-10  3.36386184e-09 -1.77629468e-10 -3.56692412e-12
 -2.37017963e-09]
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
1 93
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 3.348209e-06 
WaA: 2.0352104e-16 
WvV: 1.628519329311597e-14
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 8.020492e-07 
WaA: 1.5566705e-16 
WvV: 2.0017965589172682e-14
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 2.2845748e-09 
WaA: 1.909407e-16 
WvV: 1.54082295262

In [153]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [154]:
target

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [155]:
testRS

array([[ 5.54056374e+00,  1.41385289e+00,  7.78237543e+00,
         4.44796195e+00,  1.15785553e+01,  7.53717482e+00,
         1.38097136e+00,  1.41736010e+00,  1.99250546e+00,
        -6.78365768e-01,  5.32929901e-01,  8.07550212e-01,
        -7.84613129e-01, -1.00017962e+00,  1.37719314e-01,
         1.29712864e+00,  1.11970536e+00, -2.53847207e-04,
         5.02751337e-01, -2.42771532e-01],
       [ 4.98444877e+00,  7.16448092e+00,  1.13630646e+01,
         6.41866319e+00,  4.45323099e+00,  1.71436664e+01,
         3.88456141e+00,  2.72627402e+00,  3.37383705e+00,
         2.93318390e+00,  2.75375023e+00,  3.14028586e+00,
         1.85632797e+00,  2.59523060e+00,  3.99965325e+00,
         7.84288495e+00,  5.32985413e+00,  3.84986456e+00,
         4.69661354e+00,  4.33821627e+00],
       [ 7.56073050e+00,  1.31628233e+01,  1.47369807e+01,
         1.39259391e+01,  1.15173946e+01,  1.12157805e+01,
         8.39472018e+00,  7.56706312e+00,  1.20697379e+01,
         6.65147424e+00,  6.1

In [156]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 66.0
total testing data: 200


In [157]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [158]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

[4, 2, 5, 0, 3, 8]
[5, 2, 15, 1, 3, 16]
[2, 3, 1, 18, 8, 4, 5, 13]
[5, 1, 15, 0, 2, 6]
[9, 10, 15, 6, 12, 13]
[0, 17, 14, 3, 16]
[0, 8, 3, 5, 4, 11, 6, 18]
[4, 6, 10, 7, 8, 11, 5, 15, 13, 17]
[18, 4, 13, 19, 11, 9]
[10, 14, 18, 4, 16]
avg_accuarcy for count_0: 0.32
32


In [159]:
acc_0

32

In [160]:
total

458

# Top 1 

In [161]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [162]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

[4]
[5]
[2]
[5]
[9]
[0]
[0]
[4]
[18]
[10]


In [163]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.7 recall  0.10606060606060606


In [164]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.18421052631578946


# Top 3

In [165]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [166]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.5333333333333333 recall  0.24242424242424243


In [167]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.3333333333333333


# Top 5

In [168]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [169]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.56 recall  0.42424242424242425


In [170]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.4827586206896552


In [99]:
#User_latent_factor = loaddata['User']
#YouTuber_latent_factor = loaddata['YouTuber']

In [ ]:
np.savez('../Data/latent_factor/YRM_up10_ALL/Final/1226.npz', User=U, YouTuber=Y)

In [ ]:
def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [ ]:
softmax([-0.000000000000000000000000000000000000001,0.00000000000000000000000000000000000001])